In [12]:
# =====================================================================
# Import
# =====================================================================

# Import internal modules
import os.path
from typing import List, Set, Dict, TypedDict, Tuple, Optional

# Import 3rd party modules
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [13]:
def get_sound_files(folder: str) -> pd.DataFrame:
    """
    Function to get all sound files within a folder.
    Param: `folder` name includes background noise level and machine type (e.g. -6_db_fan)    
    Return: a DataFrame  
    """
    # Create empty lists
    sound_list: List[str] = []  # will contain all sound filenames
    sound_path_list: List[str] = []  # will contain the sound's filepaths
    noise_db_list: List[int] = [] # will contain the background noise level in db of each sound
    machine_type_list: List[str] = [] # will contain the machine type of each sound
    model_id_list: List[str] = []  # will contain the machine product id of each sound
    target_list: List[str] = [] # will contain the target value (normal=0 or anormal=1) for each sound

    # Get the relative path of the directory that contains all the sound files
    folder_path = os.path.join("assets", "sounds", folder)

    # Get all the filenames within the directory
    for path, dirs, files in os.walk(folder_path):
        for filename in files:
            # Search only filenames with the extension ".wav"
            if filename.lower().endswith(".wav"):

                # Get the filename
                sound_list.append(filename)

                # Get the filepath
                sound_path = os.path.join(path, filename)
                sound_path_list.append(sound_path)
                
                # Split filepath to retrieve the information
                path_splitted = sound_path.split("/")
                
                # Get the background noise in db
                noise_db = int(path_splitted[2].split("_")[0])
                noise_db_list.append(noise_db)
                
                # Get the machine type
                machine_type = path_splitted[2].split("_")[2]
                machine_type_list.append(machine_type)
                
                # Get the model id
                model_id = path_splitted[3].split("_")[1]
                model_id_list.append(model_id)
                
                # Get target variable (normal or anormal)
                target = path_splitted[4]
                target_list.append(target)   
    
    # Create list with the data
    data: List[float, int, str] = list(zip(noise_db_list, machine_type_list, model_id_list, sound_list, sound_path_list, target_list))
    
    # Create list with column names
    cols: List[str] = ["noise_db", "machine_type", "model_id", "sound", "sound_path", "target"]
    
    # Return a DataFrame from the lists
    return pd.DataFrame(data=data, columns=cols)

def get_all_sounds(db_list: List[int], machine_type_list: List[str]) -> pd.DataFrame:
    """
    Function to get all sound files for specified lists of background noise and machine type.
    Param: * `db_list` is a list of background noise level (i.e. -6, 0, 6)
           * `machine_type_list` is a list of machine type (i.e. fan, pump, valve, slider) 
    Return: a DataFrame  
    """
    df_list = []
    for db in db_list:
        for machine_type in machine_type_list:
            df = get_sound_files(f"{db}_db_{machine_type}")
            df_list.append(df)
            
    return pd.concat(df_list, axis=0)

In [14]:
def get_audio_features(sound_path: str) -> List[float]:
    """
    Function to extract audio features from the sound file
    Param: `sound_path` is the filepath of the sound
    Return: a list of audio features aggregated by their average values
    """
    # Read audio file
    y, sr = librosa.load(sound_path)
    
    # Get spectogram
    spect: np.ndarray = np.abs(librosa.stft(y, hop_length=512))
    spect_mean: np.float = np.mean(spect)
    
    # Get mel spectogram
    mel_spect: np.ndarray = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
    mel_spect_mean: np.float = np.mean(mel_spect)

    # Get chromagram
    chroma: np.ndarray = librosa.feature.chroma_stft(S=spect, sr=sr)
    chroma_mean: np.float = np.mean(chroma)

    # Get constant-Q chromagram
    chroma_cq: np.ndarray = librosa.feature.chroma_cqt(y=y, sr=sr)
    chroma_cq_mean: np.float = np.mean(chroma_cq)
    
    # Get chromagram cens
    chroma_cens: np.ndarray = librosa.feature.chroma_cens(y=y, sr=sr)
    chroma_cens_mean: np.float = np.mean(chroma_cens)
    
    # Get mfcc
    mfcc: np.ndarray = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_mean: np.float = np.mean(mfcc)

    # Get rms
    S: np.ndarray
    phase: np.ndarray
    S, phase = librosa.magphase(librosa.stft(y))
    rms: np.ndarray = librosa.feature.rms(S=S)
    rms_mean: np.float = np.mean(rms)
    
    # Get spectral centroid
    cent: np.ndarray = librosa.feature.spectral_centroid(y=y, sr=sr)
    cent_mean: np.float = np.mean(cent)

    # Get spectral bandwidth
    spec_bw: np.ndarray = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_bw_mean: np.float = np.mean(spec_bw)

    # Get spectral contrast
    contrast: np.ndarray = librosa.feature.spectral_contrast(S=S, sr=sr)
    contrast_mean: np.float = np.mean(contrast)

    # Get spectral flatness
    flatness: np.ndarray = librosa.feature.spectral_flatness(y=y)
    flatness_mean: np.float = np.mean(flatness)
    
    # Get roll-off frequency
    roll_off: np.ndarray = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.95)
    roll_off_mean: np.float = np.mean(roll_off)

    # tonal centroid features (tonnetz)
    tonnetz: np.ndarray = librosa.feature.tonnetz(y=y, sr=sr)
    tonnetz_mean: np.float = np.mean(tonnetz)

    # zero-crossing rate
    zero_crossing_rate: np.ndarray = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean: np.float = np.mean(zero_crossing_rate)
    
    # Return a list of audio features aggregated by their average values
    return [spect_mean, mel_spect_mean,
            chroma_mean, chroma_cq_mean, chroma_cens_mean,
            mfcc_mean, rms_mean, cent_mean, 
            spec_bw_mean, contrast_mean, flatness_mean,
            roll_off_mean, tonnetz_mean, zero_crossing_rate_mean]

In [57]:
# df = get_all_sounds([-6, 0, 6], ["fan", "valve", "pump", "slider"])
df = get_all_sounds([-6], ["fan"])
df.head()

,noise_db,machine_type,model_id,sound,sound_path,target
0,-6,fan,00,00000059.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000005...,abnormal
1,-6,fan,00,00000071.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000007...,abnormal
2,-6,fan,00,00000065.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000006...,abnormal
3,-6,fan,00,00000273.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000027...,abnormal
4,-6,fan,00,00000267.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000026...,abnormal


In [48]:
df.shape

(5550, 6)

In [19]:
# Create list with column names
audio_features_cols: List[str] = ["spectogram", "mel_spectogram",
                   "chromagram", "chromagram_cq", "chromagram_cens",
                   "mfcc", "rms", "spectral_centroid", 
                   "spectral_bandwidth", "spectral_contrast", "spectral_flatness",
                   "roll_off_frequency", "tonnetz", "zero_crossing_rate"]

df[audio_features_cols] = df[["sound_path"]].apply(lambda x: pd.Series(get_audio_features(x.sound_path)), axis=1)

KeyboardInterrupt: 

In [ ]:
# Create list with column names
audio_features_cols: List[str] = ["spectogram", "mel_spectogram",
                   "chromagram", "chromagram_cq", "chromagram_cens",
                   "mfcc", "rms", "spectral_centroid", 
                   "spectral_bandwidth", "spectral_contrast", "spectral_flatness",
                   "roll_off_frequency", "tonnetz", "zero_crossing_rate"]

# Create empty columns
for col in audio_features_cols:
    df[col] = np.nan

for idx, sound in enumerate(df.sound_path.values):
    feature_values_list = get_audio_features(sound)
#     print(feature_values_list)
    for i in range(len(feature_values_list)):
#         print(idx, str(audio_features_cols[i]), feature_values_list[i])
        df[audio_features_cols[i]].iloc[idx] = feature_values_list[i]

In [34]:
df_test.head()

,noise_db,machine_type,model_id,sound,sound_path,target,spectogram,mel_spectogram,chromagram,chromagram_cq,chromagram_cens,mfcc,rms,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,roll_off_frequency,tonnetz,zero_crossing_rate
0,-6,fan,00,00000059.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000005...,abnormal,0.087980,0.034124,0.519510,0.348788,0.243607,-4.044741,0.007393,1180.264031,1579.000699,24.305225,0.000133,5154.829000,0.042503,0.049593
1,-6,fan,00,00000071.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000007...,abnormal,0.100928,0.039813,0.753670,0.389851,0.253925,-3.492745,0.007938,1036.513080,1375.484191,21.282001,0.000112,4208.891873,0.050504,0.037650
2,-6,fan,00,00000065.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000006...,abnormal,0.089889,0.036545,0.533704,0.351515,0.242565,-3.905537,0.007585,1271.447206,1735.250822,24.272330,0.000149,5743.070145,0.043719,0.049436
3,-6,fan,00,00000273.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000027...,abnormal,0.090242,0.025801,0.730573,0.508074,0.269959,-3.973263,0.006392,1050.479779,1347.219254,21.138120,0.000141,4106.271922,-0.022377,0.044327
4,-6,fan,00,00000267.wav,assets/sounds/-6_db_fan/id_00/abnormal/0000026...,abnormal,0.087292,0.022661,0.583526,0.484912,0.270946,-4.673796,0.005993,1414.325444,1763.148493,23.825591,0.000246,5714.992047,-0.038684,0.066287


In [39]:
get_audio_features(df_test.sound_path.iloc[2])

[0.08988863,
 0.03654531,
 0.53370386,
 0.35151482,
 0.24256478440084428,
 -3.9055371,
 0.007584500401487915,
 1271.4472060240762,
 1735.2508219906167,
 24.272329736071338,
 0.00014936722,
 5743.070144830336,
 0.0437188022909946,
 0.04943649398201856]

In [9]:
df.head()

,noise_db,machine_type,model_id,sound,sound_path,target,spectogram,mel_spectogram,chromagram,chromagram_cq,chromagram_cens,mfcc,rms,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,roll_off_frequency,tonnetz,zero_crossing_rate
0,-6,slider,00,00000059.wav,assets/sounds/-6_db_slider/id_00/abnormal/0000...,abnormal,0.103656,0.028626,0.508862,0.514267,0.264862,-5.768414,0.006748,1746.776097,1955.708303,24.889785,0.000356,6312.725788,-0.014067,0.077769
1,-6,slider,00,00000071.wav,assets/sounds/-6_db_slider/id_00/abnormal/0000...,abnormal,0.102113,0.025405,0.592927,0.509345,0.265619,-5.547102,0.006390,1762.671976,1924.965304,24.371674,0.000404,6249.724931,-0.026640,0.080915
2,-6,slider,00,00000065.wav,assets/sounds/-6_db_slider/id_00/abnormal/0000...,abnormal,0.103404,0.029101,0.615363,0.519190,0.264350,-5.850099,0.006801,1797.617686,2018.689316,23.995691,0.000357,6408.051429,-0.015122,0.074452
3,-6,slider,00,00000273.wav,assets/sounds/-6_db_slider/id_00/abnormal/0000...,abnormal,0.120839,0.029576,0.674066,0.608943,0.279228,-4.685860,0.006914,1693.404780,1843.512061,21.976415,0.000446,5747.141969,-0.006995,0.082713
4,-6,slider,00,00000267.wav,assets/sounds/-6_db_slider/id_00/abnormal/0000...,abnormal,0.114762,0.032621,0.658467,0.624440,0.276571,-4.298293,0.007232,1503.216934,1768.753095,21.985056,0.000396,5490.966797,-0.000406,0.065476


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   noise_db            100 non-null    int64  
 1   machine_type        100 non-null    object 
 2   model_id            100 non-null    object 
 3   sound               100 non-null    object 
 4   sound_path          100 non-null    object 
 5   target              100 non-null    object 
 6   spectogram          100 non-null    float64
 7   mel_spectogram      100 non-null    float64
 8   chromagram          100 non-null    float64
 9   chromagram_cq       100 non-null    float64
 10  chromagram_cens     100 non-null    float64
 11  mfcc                100 non-null    float64
 12  rms                 100 non-null    float64
 13  spectral_centroid   100 non-null    float64
 14  spectral_bandwidth  100 non-null    float64
 15  spectral_contrast   100 non-null    float64
 16  spectral_

In [11]:
df.isnull().any()

noise_db              False
machine_type          False
model_id              False
sound                 False
sound_path            False
target                False
spectogram            False
mel_spectogram        False
chromagram            False
chromagram_cq         False
chromagram_cens       False
mfcc                  False
rms                   False
spectral_centroid     False
spectral_bandwidth    False
spectral_contrast     False
spectral_flatness     False
roll_off_frequency    False
tonnetz               False
zero_crossing_rate    False
dtype: bool